Import Libraries

In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd

Get all the property addresses

In [3]:
# rightmove main URL
### initialise search ### 
URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=" + str(i) + "&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


10  Pages available
Number of links acquired:  219
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/107409182#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/109937678#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/110514809#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114573920#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115021484#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115645691#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106038#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106077#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116108690#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116233709#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116243063#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116491958#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/propertie

Fishing Properties

In [4]:
i = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")
    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    typeOfProperty = soup.select('p')[1].text
    noOfBeds = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', cleanIds[i])
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            foundFloorPlanImage = 'None'
            pass
    
    data = [str(price), str(propertyAddress), str(typeOfProperty), str(noOfBeds), str(foundFloorPlanImage), str(mapLocation)]
    all_info = np.append(all_info, data)
    i = i+1 
    end = time()
    print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!')
    
# split the array into equally divided arrays
all_info = np.split(all_info, len(cleanLinks))
print(all_info)

1 out of 219 It took 2.01 seconds!
2 out of 219 It took 0.56 seconds!
3 out of 219 It took 0.49 seconds!
4 out of 219 It took 1.67 seconds!
5 out of 219 It took 0.53 seconds!
6 out of 219 It took 0.58 seconds!
7 out of 219 It took 0.52 seconds!
8 out of 219 It took 0.51 seconds!
9 out of 219 It took 0.5 seconds!
10 out of 219 It took 0.47 seconds!
11 out of 219 It took 0.57 seconds!
12 out of 219 It took 0.48 seconds!
13 out of 219 It took 0.62 seconds!
14 out of 219 It took 1.77 seconds!
15 out of 219 It took 0.55 seconds!
16 out of 219 It took 0.46 seconds!
17 out of 219 It took 0.55 seconds!
18 out of 219 It took 0.48 seconds!
19 out of 219 It took 0.54 seconds!
20 out of 219 It took 0.48 seconds!
21 out of 219 It took 0.46 seconds!
22 out of 219 It took 0.52 seconds!
23 out of 219 It took 0.55 seconds!
24 out of 219 It took 0.49 seconds!
25 out of 219 It took 0.51 seconds!
26 out of 219 It took 0.48 seconds!
27 out of 219 It took 0.6 seconds!
28 out of 219 It took 0.5 seconds!
29 o

Image Processing

In [129]:
import importlib 
from numpy import empty
from searchFunction import search 

updated_info = np.array([])

for info in range(len(all_info)):
    #print(str(info)+' out of '+str(len(all_info)))
    
    if not all_info[info][4] == 'None':
        # Load the img
        req = urllib.request.urlopen(all_info[info][4])
        arr = np.asarray(bytearray(req.read()))
        img = cv2.imdecode(arr, -1) # 'Load it as it is'

        # Display loaded image
        # cv2.imshow('Floor Plan', img)
        # if cv2.waitKey() & 0xff == 27: quit()

        # OCR
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        configs = r'--psm 6'
        try:
            rawText = pytesseract.image_to_string(img, config=configs)
        except:
            continue
        rawText = rawText.lower()
        rawText = re.sub(r',', '', str(rawText)) # removes all commas 
        rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
        #print(rawText)

        newText = search(rawText)
        #print(newText)
        newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
        updated_info = np.append(updated_info, newInfo)

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/7) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'typeOfProperty', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'area in ft'])
print(updated_info)

0 0 0 0 0
1278 0 0 0 0
1169 0 0 0 0
1695 0 0 0 0
1398 0 0 0 0
1519 0 1519 0 0
0 1187 0 0 0
1545 0 0 0 0
1651 0 0 0 0
0 0 0 1170 0
0 0 0 0 0
961 0 0 0 0
0 0 0 0 0
893 0 0 0 0
765 0 0 0 0
988 0 0 0 0
0 0 0 1099 0
1039 0 0 0 0
853 0 0 0 0
0 974 0 0 0
1498 0 0 0 0
0 970 0 0 0
1178 0 0 0 0
1270 0 1270 0 0
1485 0 0 0 0
814 0 0 0 0
1291 0 0 0 0
940 0 0 0 0
865 0 0 0 0
1200 0 0 0 0
715 0 715 0 0
768 0 0 0 0
0 0 0 0 781
1139 0 0 0 0
0 0 0 0 0
470 0 470 0 0
1054 0 0 0 0
0 0 0 0 0
0 0 0 0 0
766 0 0 0 0
831 0 0 0 0
877 0 0 0 0
0 1182 0 0 0
425 0 0 0 0
0 0 0 0 0
867 0 0 0 0
775 0 0 0 0
879 0 0 0 0
0 0 0 0 0
0 1102 0 0 0
1027 0 0 0 0
1247 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 726
1238 0 0 0 0
642.0 419 0 0 1088
1077 0 0 0 0
0 0 0 0 0
0 0 0 742 0
382 0 0 0 0
753 0 0 0 0
0 775 0 0 0
1114 0 1114 0 0
0 0 0 868 868
0 0 0 0 0
685 0 0 0 0
0 0 0 0 0
1481 0 0 0 0
0 0 0 0 0
971 0 0 0 0
750 0 0 0 0
0 765 0 0 0
1136 0 1136 0 0
815 0 0 0 0
0 0 0 0 0
1124 0 0 0 0
506 0 506 0 0
814 0 0 0 0
814 0 0 0 0
0 0 

In [127]:
# this should load the old file and append any new data found to it making sure it's all unique data
# and not repeated 

updated_info.to_csv('output.csv', index=False)

A good way to filter the area in a better way is to convert all strings into numbers and then extract the largest number as the "area" in ft...